# Abordagem 5

Usando a abordagem 5 para gerar templates com foco em templates positivos e negativos. Uma possível aplicação seria testar a capacidade linguística "Vocabulary" com o teste MFT.

As etapas desta abordagem são:

1. Quebrar as instâncias em sentenças
2. Rankear as palavras de cada sentença
3. Filtrar as sentenças pelo tamanho (maior ou igual a 5 palavras)
4. Filtrar sentenças com palavras relevantes (verbos ou adjetivos)
5. Filtrar sentenças com alta confiança na predição das palavras relevantes da etapa anterior
6. Substituir as palavras relevantes por máscaras

In [1]:
%config Completer.use_jedi = False
import sys
sys.path.append('../')

## Carregando o dataset, o modelo alvo e os modelos auxiliares

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

imdb_df = pd.read_csv('./data/imdb_sampled/data-100samples.csv')
imdb_df.head(5)

,label,text,words
0,1,"Christian Duguay directed this tidy little espionage thriller early in his career. It plays on TV pretty regularly, albeit with some terrific scenes of violence and sex unfortunately trimmed. I finally got around to seeing the theatrical version on a $3 tape from the local video store. Naval officer Aidan Quinn is recruited to impersonate the notorious Carlos the Jackal, and gets a little too caught up in the role. Donald Sutherland Ben Kingsley play Quinn's superiors, with Sutherland a true zealot and Kingsley as the more level-headed one. The first half of this fun flick shows Quinn being trained and indoctrinated. The second half has him out in the field, making love to the Jackal's woman and shooting it out with sundry enemies. The idea is to make the Jackal look like a turncoat to the Russians, and let them take care of the world's most notorious assassin. Things don't exactly play out as planned. At times, I almost expected the cast to break out laughing at some of the corny dialogue, but they all play it very straight. In the end, this is one terrific little thriller that deserves your attention. The Jackal's former mistress teaching the highly proper and very married Quinn to rough her up, lick blood from her face, and then go down on her, alone is worth the price of admission.",227
1,1,"New Yorkers contemporaneous with this film will recall how reflective of its time it is and how well cast and crew captured America, New York City of that era.<br /><br />Norman Wexler's script delineates the different worlds the various sub groupings live in and Avildsen's direction brings out phenomenal performances all around. Peter Boyle's prodigious talent is on display as never before nor since. Clearly it is the best character portrayal the always likable Dennis Patrick ever accomplished.<br /><br />What I will always remember about JOE is the feeling of having been in a virtual state of shock coming out of the theater. Knowing that what the screen portrayed was seething under the surface in neighborhoods throughout the five boroughs of the City of New York.<br /><br />This film needs to be remembered.",133
2,0,"I love oddball animation, I love a lot of Asian films, but I didn't love this particular product of Japan. The Fuccons are supposedly an American family (they're all mannequins) who have moved to Japan, and they're somewhat a 50's sitcom type family, with slightly more modern sensibilities at times. The DVD features several very short episodes (like less than 5 minutes each?) and I did not find it to be either funny or entertaining, not even in a weird way. I'm not sure what the appeal is of this. I did pick up on some satire here and there, gosh, who wouldn't, but satire is usually somewhat humorous, isn't it? And nothing I saw or heard rated even a little smirk. I picked this up used and it certainly SOUNDED appealing, but I guess either I'm missing the point or it's just plain LAME. The box even says it's Fuccon hilarious, right there on the front, but I beg to differ. 2 out of 10.",166
3,1,"I have seen this film probably a dozen times since it was originally released theatrically. Anyone who calls this movie trash or horrible just doesn't understand action films or recognize a good one. Perhaps to some the incidents and outcomes may seem far fetched, but in my opinion screenwriter Shane Black ( Lethal Weapon/ Kiss Kiss Bang Bang) crafted one of the most well thought out action adventures you will ever come across. Over the top or not this film flows like clockwork and the action just keeps coming. The final action sequence is one of the best I have ever seen in any film. The cast in this film crackles. Genna Davis gave a tremendous performance and its a damn shame there was never a ""LKG"" sequel. Samuel L. Jackson is hilarious as her sidekick Mitch a down on his luck private eye trying to help her discover her lost past and make a few bucks. If Baffles me how anyone could not like this film. I

In [3]:
import re
import numpy as np
from torch.nn.functional import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def pre_proccess(text):
    text = text.lower()
    text = re.sub('["\',!-.:-@0-9/]()', ' ', text)
    return text

# Wrapper to adapt output format
class SentimentAnalisysModelWrapper:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        
    def __predict(self, text_input):
        text_preprocessed = pre_proccess(text_input)
        tokenized = self.tokenizer(text_preprocessed, padding=True, truncation=True, max_length=512, 
                                    add_special_tokens = True, return_tensors="pt")
        
        tensor_logits = self.model(**tokenized)
        prob = softmax(tensor_logits[0]).detach().numpy()
        pred = np.argmax(prob)
        
        return pred, prob
    
    def predict_label(self, text_inputs):
        return self.predict(text_inputs)[0]
        
    def predict_proba(self, text_inputs):
        return self.predict(text_inputs)[1]
        
    def predict(self, text_inputs):
        if isinstance(text_inputs, str):
            text_inputs = [text_inputs]
        
        preds = []
        probs = []

        for text_input in text_inputs:
            pred, prob = self.__predict(text_input)
            preds.append(pred)
            probs.append(prob[0])

        return np.array(preds), np.array(probs) # ([0, 1], [[0.99, 0.01], [0.03, 0.97]])

# Auxiliar function to load and wrap a model from Hugging Face
def load_model(model_name):
    print(f'Loading model {model_name}...')
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    return SentimentAnalisysModelWrapper(model, tokenizer)

# Hugging Face hosted model names 
imdb_models = {
    'bert': 'textattack/bert-base-uncased-imdb', 
    'albert': 'textattack/albert-base-v2-imdb', 
    'distilbert': 'textattack/distilbert-base-uncased-imdb', 
    'roberta': 'textattack/roberta-base-imdb', 
    'xlnet': 'textattack/xlnet-base-cased-imdb'
}

In [4]:
m1 = load_model(imdb_models['albert'])
m2 = load_model(imdb_models['distilbert'])
m3 = load_model(imdb_models['roberta'])
m4 = load_model(imdb_models['xlnet'])

# Models to be used as oracle
models = [m1, m2, m3, m4]
# Target model
model = load_model(imdb_models['bert'])

Loading model textattack/albert-base-v2-imdb...
Loading model textattack/distilbert-base-uncased-imdb...
Loading model textattack/roberta-base-imdb...


Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model textattack/xlnet-base-cased-imdb...
Loading model textattack/bert-base-uncased-imdb...


# Gerando os templates
O método de rankeamento das palavras usado no PosNegTemplateGenerator é o Replace-1 Score

In [5]:
from template_generator.tasks.sentiment_analisys import PosNegTemplateGeneratorApp5

### Número inicial de instâncias: 5

In [6]:
# Sampling instances
np.random.seed(220)
n_instances = 5
df_sampled = imdb_df.sample(n_instances)

instances = [x for x in df_sampled['text'].values]

In [7]:
tg = PosNegTemplateGeneratorApp5(model, models)
templates = tg.generate_templates(instances, n_masks=2, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 40 sentences were generated.
Ranking words using Replace-1 Score...


/tmp/ipykernel_2473/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Word ranking done.
Filtering instances by contaning a minimmum of words: 5...
:: 38 sentences remaining.
Filtering instances by relevant words...
:: 11 sentences remaining.
Filtering instances by relevant words classification score greater than 0.8
:: 2 sentences remaining.
Predicting inputs...
:: Sentence predictions done.


#### Tempo de execução para 5 instâncias: 1m 15.8s
filipe: 41.6s

In [8]:
df = tg.to_dataframe()
df

,label,original_text,masked_text,template_text
0,1,"Just forget the itty-bitty disappointments, like the fact that there were only adults in this movie based on a pup's point of view, because that's just 0.5% or less of the movie's wonderful effect on the viewer.","Just {mask} the itty - bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {mask} effect on the viewer .","Just {neg_verb} the itty - bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {pos_adj} effect on the viewer ."
1,1,"Great music, lively acting, talented animation and directing, and a powerful storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family.","{mask} music , lively acting , talented animation and directing , and a {mask} storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family .","{pos_adj} music , lively acting , talented animation and directing , and a {pos_adj} storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family ."


In [9]:
tg.lexicons

{'pos_verb': [],
 'neg_verb': ['forget'],
 'pos_adj': ['Great', 'wonderful', 'powerful'],
 'neg_adj': []}

### Número inicial de instâncias: 100

In [10]:
# Using all 100 instances
instances = [x for x in imdb_df['text'].values]

In [11]:
%%time
# 14min 55.2s
tg = PosNegTemplateGeneratorApp5(model, models)
templates = tg.generate_templates(instances, n_masks=2, ranked_words_count=4, min_classification_score=0.8)

Converting texts to sentences...
:: 852 sentences were generated.
Ranking words using Replace-1 Score...


/tmp/ipykernel_2473/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


:: Word ranking done.
Filtering instances by contaning a minimmum of words: 5...
:: 806 sentences remaining.
Filtering instances by relevant words...
:: 207 sentences remaining.
Filtering instances by relevant words classification score greater than 0.8
:: 31 sentences remaining.
Predicting inputs...
:: Sentence predictions done.
CPU times: user 2h 17min 4s, sys: 17 s, total: 2h 17min 21s
Wall time: 14min 55s


#### Tempo de execução para 100 instâncias: 28m 41.8s

In [12]:
df_templates = tg.to_dataframe()
df_templates.insert(0, "template_index", df_templates.index.map(lambda x: int(x)+1))
df_templates

,template_index,label,original_text,masked_text,template_text
0,1,0,"Directed, co-written, co-produced and co-edited with dumbfounding maladroitness by Melanie Anne Phillips, acted with dismaying flatness by a rank no-name cast, further marred by lethargic pacing, a drably meandering narrative, murky, under-lit, eye-straining cinematography, a shivery, redundantly thudding pseudo-John Carpenter synthesizer score, and a cruddy, herky-jerky stop motion animation wormoid thingie that's only quickly glimpsed at the very end of the movie, this extremely clunky, amateurish and hence quite delectably dreadful would-be scarefest commits all the necessary bad film missteps to qualify as a real four-star stinkeroonie.","Directed , co - written , co - produced and co - edited with {mask} maladroitness by Melanie Anne Phillips , acted with dismaying flatness by a rank no - name cast , further marred by lethargic pacing , a drably meandering narrative , murky , under - lit , eye - straining cinematography , a shivery , redundantly thudding pseudo - John Carpenter synthesizer score , and a cruddy , herky - jerky stop motion animation wormoid thingie that 's only quickly glimpsed at the very end of the movie , this extremely clunky , {mask} and hence quite delectably dreadful would - be scarefest commits all the necessary bad film missteps to qualify as a real four - star stinkeroonie .","Directed , co - written , co - produced and co - edited with {neg_adj} maladroitness by Melanie Anne Phillips , acted with dismaying flatness by a rank no - name cast , further marred by lethargic pacing , a drably meandering narrative , murky , under - lit , eye - straining cinematography , a shivery , redundantly thudding pseudo - John Carpenter synthesizer score , and a cruddy , herky - jerky stop motion animation wormoid thingie that 's only quickly glimpsed at the very end of the movie , this extremely clunky , {neg_adj} and hence quite delectably dreadful would - be scarefest commits all the necessary bad film missteps to qualify as a real four - star stinkeroonie ."
1,2,1,"Just forget the itty-bitty disappointments, like the fact that there were only adults in this movie based on a pup's point of view, because that's just 0.5% or less of the movie's wonderful effect on the viewer.","Just {mask} the itty - bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {mask} effect on the viewer .","Just {neg_verb} the itty - bitty disappointments , like the fact that there were only adults in this movie based on a pup 's point of view , because that 's just 0.5 % or less of the movie 's {pos_adj} effect on the viewer ."
2,3,1,"Great music, lively acting, talented animation and directing, and a powerful storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family.","{mask} music , lively acting , talented animation and directing , and a {mask} storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family .","{pos_adj} music , lively acting , talented animation and directing , and a {pos_adj} storyline tell a tale of the difference between freedom on the streets of New England and a cozy home with your family ."
3,4,1,But not only are messages great- its hilarious!,But not only are messages great- {mask} {mask} !,But not only are messages great- {neg_verb} {pos_adj} !
4,5,1,"I can't say that this movie deserves a ten, because I would be lying.","I ca n't say that this movie {mask} a ten , because I would be {mask} .","I ca n't say that this movie {pos_verb} a ten , because I would be {neg_verb} ."
5,6,1,It carries the unique bond of a friendship between a teenager and a dolphin.,It {mask} the {mask} bond of a friendship between a teenager and a dolphin .,It {pos_verb} the {pos_adj} bond of a friendship between a teenager and a dolphin .
6,

In [13]:
df_templates.to_csv("generated_templates/generated_templates_approach5.csv")

In [14]:
print(tg.lexicons)

{'pos_verb': ['watching', 'enjoy', 'deserves', 'celebrating', 'shares', 'watched', 'sit', 'loved', 'carries'], 'neg_verb': ['its', 'wastes', 'mutilating', 'had', 'botched', 'forget', 'supposed', 'pretending', 'replaces', 'lying'], 'pos_adj': ['greener', 'modern', 'great', 'sexy', 'tender', 'Great', 'enjoyable', 'strong', 'good', 'imaginative', 'wonderful', 'Good', 'unique', 'hilarious', 'powerful'], 'neg_adj': ['silly', 'amateurish', 'Bad', 'dull', 'bad', 'shambolic', 'stupid', 'boring', 'dumbfounding', 'ridiculous', 'tortuous', 'forgettable', 'absurd', 'involved', 'sick', 'awful', 'faulty', 'judgmental', 'homophobic', 'meaner']}


# Usando os templates gerados pelo TemplateGenerator no CheckList

In [15]:
from checklist.editor import Editor
from checklist.test_suite import TestSuite
from checklist.test_types import MFT

In [16]:
lexicons = tg.lexicons
templates = tg.template_texts
masked = tg.masked_texts
labels = [sent.prediction.label for sent in tg.sentences]

editor = Editor()
editor.add_lexicon('pos_verb', lexicons['pos_verb'])
editor.add_lexicon('neg_verb', lexicons['neg_verb'])
editor.add_lexicon('pos_adj', lexicons['pos_adj'])
editor.add_lexicon('neg_adj', lexicons['neg_adj'])

suite = TestSuite()

In [17]:
for template, label, i in zip(templates, labels, range(len(templates))):
    t = editor.template(template, remove_duplicates=True, labels=int(label))

    suite.add(MFT(
        data=t.data,
        labels=label,
        capability="Vocabullary", 
        name=f"Test: MFT with vocabullary - template{i+1}",
        description="Checking if the model can handle vocabullary"))

In [18]:
suite.run(model.predict, overwrite=True)
suite.save('./suites/posneg-approach5.suite')

Running Test: MFT with vocabullary - template1
Predicting 20 examples


/tmp/ipykernel_2473/1179770237.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = softmax(tensor_logits[0]).detach().numpy()


Running Test: MFT with vocabullary - template2
Predicting 150 examples
Running Test: MFT with vocabullary - template3
Predicting 15 examples
Running Test: MFT with vocabullary - template4
Predicting 150 examples
Running Test: MFT with vocabullary - template5
Predicting 90 examples
Running Test: MFT with vocabullary - template6
Predicting 135 examples
Running Test: MFT with vocabullary - template7
Predicting 20 examples
Running Test: MFT with vocabullary - template8
Predicting 10 examples
Running Test: MFT with vocabullary - template9
Predicting 9 examples
Running Test: MFT with vocabullary - template10
Predicting 150 examples
Running Test: MFT with vocabullary - template11
Predicting 300 examples
Running Test: MFT with vocabullary - template12
Predicting 20 examples
Running Test: MFT with vocabullary - template13
Predicting 135 examples
Running Test: MFT with vocabullary - template14
Predicting 15 examples
Running Test: MFT with vocabullary - template15
Predicting 15 examples
Running T

# Carregando suite de teste

In [19]:
from checklist.test_suite import TestSuite
suite = TestSuite.from_file('./suites/posneg-approach5.suite')

# suite.visual_summary_table()

In [20]:
passed = 0
failed = 0
for test_name in suite.tests:
    table = suite.visual_summary_by_test(test_name)
    
    failed += table.stats['nfailed']    
    passed += table.stats['npassed']
    assert table.stats['nfailed'] + table.stats['npassed'] == len(table.filtered_testcases)

print(f"{failed = } ({(failed/(passed+failed))*100:.2f}%)")
print(f"{passed = } ({(passed/(passed+failed))*100:.2f}%)")
print(f"total = {passed+failed}")
print("templates:", len(suite.tests))

failed = 541 (15.85%)
passed = 2872 (84.15%)
total = 3413
templates: 31


In [21]:
table = suite.visual_summary_by_test('Test: MFT with vocabullary - template3')

for item in table.candidate_testcases:
    print(item['examples'][0]['new']['text'])